In [ ]:
pip install numpy pandas matplotlib scikit-learn

In [1]:
import sys
import random
from functools import reduce
import operator
import warnings
import numpy as np
import time
import pandas as pd
import atexit
import queue
import random
import concurrent.futures
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

np.set_printoptions(suppress=True)
np.set_printoptions(threshold=sys.maxsize)
from matplotlib.pyplot import figure
import collections
from collections import OrderedDict
import functools
import operator
from threading import Barrier

################################################################
random.seed(10) 
bit_num = 32

################################################################
def generate_groups(client_num, min_group_size, max_group_size, min_groups):
    # If clients are fewer than the specified minimum groups of minimum size
    if client_num < min_groups * min_group_size:
        return [client_num]
    
    groups = []
    remaining_clients = client_num

    while remaining_clients > 0:
        if len(groups) < min_groups - 1:  # ensure we have the minimum number of groups
            upper_limit = min(remaining_clients - (min_groups - len(groups) - 1) * min_group_size, max_group_size)
        else:
            upper_limit = min(remaining_clients, max_group_size)
        
        # Adjust if the calculated upper limit is below the min_group_size
        if upper_limit < min_group_size:
            smallest_group_index = groups.index(min(groups))
            groups[smallest_group_index] += remaining_clients
            break
        
        group_size = random.randint(min_group_size, upper_limit)
        groups.append(group_size)
        remaining_clients -= group_size

    return groups

def generate_secret_share():
    return random.randint(0, 1)

def secret_share_binary_list(binary_list, k):
    shares = [[] for _ in range(k)]

    for binary_number in binary_list:
        generated_shares = [generate_secret_share() for _ in range(k-1)]
        last_share = binary_number ^ reduce(operator.xor, generated_shares)
        generated_shares.append(last_share)

        for i, share in enumerate(generated_shares):
            shares[i].append(share)

    return shares

def share(secret, client_num):
    shares = [0]*client_num
    for i in range(client_num - 1):
        shares[i] = random.randint(1, 6)
    shares[client_num - 1] = secret - sum(shares[:client_num - 1])
    return shares

def calculate_r_prime(b):
    m = len(b)
    r_prime = 0

    for i in range(m):
        r_prime += 2**(m - 1 - i) * b[i]
    
    return r_prime

def bitwise_add_and_carry(a, b):
    # Ensure the binary vectors are the same length by padding the shorter one with zeros
    if len(a) < len(b):
        a = [0] * (len(b) - len(a)) + a
    elif len(b) < len(a):
        b = [0] * (len(a) - len(b)) + b

    carry = 0
    sum = [0] * len(a)
    for i in reversed(range(len(a))):
        # Bitwise addition
        sum[i] = a[i] ^ b[i] ^ carry

        # Update carry
        carry = (a[i] & b[i]) | (carry & (a[i] ^ b[i]))

    final_carry = carry

    return sum, final_carry

def share_bit(input_, k):
    shares = [random.randint(0,1) for _ in range(k-1)]
    last_share = input_ ^ reduce(operator.xor, shares)
    shares.append(last_share)
    return tuple(shares)

def share_list(input_list, client_num):
    all_shares = [ [0]*len(input_list) for _ in range(client_num)]
    for idx, num in enumerate(input_list):
        shares = share_n(num, client_num)
        for j, share in enumerate(shares):
            all_shares[j][idx] = share
    return all_shares

################################################################

def share_n(secret, client_num):
    shares = [0]*client_num
    remaining = secret
    for i in range(client_num - 1):
        max_share = max(1, remaining - (client_num - i - 1))
        shares[i] = random.randint(1, max_share)
        remaining -= shares[i]
    shares[client_num - 1] = remaining
    return shares

def share_beta(input_, client_num):
    shares = [[0]*len(input_) for _ in range(client_num)]
    
    # Generate random shares for each client (except the last one)
    for i in range(client_num - 1):
        shares[i] = [random.randint(1, 6) for _ in range(len(input_))]
    
    # The last client's share is the difference between the input and sum of previous shares
    shares[client_num - 1] = [x - sum(share[i] for share in shares) for i, x in enumerate(input_)]
    
    return shares

################################################################

def CP(i, x, y, U, V, W, r_l, r_l_bits_, 
       r_p_l, r_p_l_bits_, sum_r_rp_bits_, carry, rand_bit_p, r_, s_, 
       r_p_mod2_share, r_p_mod2_num, r_p_mod2_bit_0, r_dp_mod2_1,input_queue, output_queue,barrier):    
    
    def mul_number(i, s, d):
        D = s - U
        E = d - V

        for q in output_queue:
            q.put((D, E))

        DE_values = [(D, E)] + [input_queue.get() for _ in range(len(output_queue))]

        D_sum = sum(D for D, E in DE_values)
        E_sum = sum(E for D, E in DE_values)

        Z = W + (D_sum * V) + (U * E_sum)

        if i == 0:
            Z += ((D_sum * E_sum))
        return Z

    def A2B(s):
        y = s%2 
        return y

    def B2A(i, s):

        r = rand_bit_p
        r_2 = A2B(r)
        c_1 = (s ^ r_2)

        for q in output_queue:
            q.put(c_1)

        final_c = c_1

        for _ in range(len(output_queue)):
            final_c = final_c ^ input_queue.get()

        c = final_c

        if i==0 :
            s = c + r - (2 * c * r)
        else:
            s = r - (2 * c * r)

        return s

    def PreMulC(i, a):

    #     if i == 0: 
    #         print(f'Thread {i} - U: {U}')

        def MulModList(lst):
            result = 1
            for e in lst:
                result = (result * e) 
            return result

        k = len(a)
        p = [None] * k

        r = r_
        s = s_

        u_1 = []

        for p in range(k):
            result2 = mul_number(i, r[p], s[p])
            u_1.append(result2)

        for q in output_queue:
            q.put(u_1)

        u_1_total = u_1

        for _ in range(len(output_queue)):
            u_other = input_queue.get()
            u_1_total = [sum(t) for t in zip(u_1_total, u_other)]

        u = u_1_total

        v = [mul_number(i, r[f+1], s[f]) for f in range(k-1)]

        w = [None] * k
        z = [None] * k
        p = [None] * k

        w[0] = r[0]

        temp_list = []
        for t in u:
            temp_list.append(1 / t)

        for t in range(1, k):
            w[t] = v[t-1] * temp_list[t-1]

        temp_list = []
        for t in u:
            temp_list.append(1 / t)

        for g in range(k):
            z[g] = s[g] * temp_list[g]

        m_1 = [mul_number(i, w[g], a[g]) for g in range(k)]

        for q in output_queue:
            q.put(m_1)

        m_1_total = m_1
        for _ in range(len(output_queue)):
            m_other = input_queue.get()
            m_1_total = [sum(t) for t in zip(m_1_total, m_other)]

        m = m_1_total

        p[0] = a[0]

        for j in range(1, k):
            b = MulModList(m[:j+1])
            p[j] = (z[j] * b)

        return p


    def Mod2(i , a, k):

        if i==0:
            c_1 = (2**(k-1)) + a + (2 * r_dp_mod2_1) + r_p_mod2_bit_0
        else:
            c_1 = a + (2 * r_dp_mod2_1) + r_p_mod2_bit_0

        for q in output_queue:
            q.put(c_1)

        c = c_1 + sum(input_queue.get() for _ in range(len(output_queue)))

        c_0 = c % 2

        if i ==0 :
            a_0 = c_0 + r_p_mod2_bit_0 - (2*c_0 * r_p_mod2_bit_0)
        else:
            a_0 = r_p_mod2_bit_0 - (2*c_0 * r_p_mod2_bit_0)

        return a_0

    def PreOrC(i, a):

    #     if i == 0: 
    #         print(f'Thread {i} - U: {U}')

        k = len(a)

        p = [0] * k

        if i==0:
            b = [t + 1 for t in a]
        else:
            b = a

    #     print(f'z{i} = {b}')

        b_result = PreMulC(i, b)

    #     b_result = [round(num) for num in b_result]
    #     print(f'z{i} = {b_result}')

        p[0] = a[0]

        for e in range(1, k):
            mod_ = Mod2(i, b_result[e], k)
            if i==0:          
                p[e] = 1 - mod_
            else:
                p[e] = - mod_

        num = p
        p = [round(num) if abs(num - round(num)) < 0.5 else round(num + 0.5) for num in num]
        return p

    def LTBits(i, R_int, e):

        R_tmp = R_int
        
        y = [0] * len(e)
        z = [0] * len(e)
        w = [0] * len(e)
        c = 0
        cc = 0

        for g in range(len(e)):
            if i == 0:
                y[len(e) - 1 - g] = e[len(e) - 1 - g] ^ (R_tmp % 2)
            else:
                y[len(e) - 1 - g] = e[len(e) - 1 - g]
            R_tmp >>= 1

        R_tmp = R_int 

        k = len(y)

        b2a_list = []
        for h in range(k):
            temp_b2a = B2A(i, y[h])
            b2a_list.append(temp_b2a)

        y = b2a_list

        z = PreOrC(i, y)

        a2b_list = []
        for m in range(k):
            a2b_ = A2B(z[m])
            a2b_list.append(a2b_)

        z = a2b_list

        w[0] = z[0]

        for g in range(len(z) - 1):
            w[len(z) - 1 - g] = (z[len(z) - 1 - g] ^ z[len(z) - 2 - g])

        for h in range(len(w)):
            c += int(w[len(w) - 1 - h] & (R_tmp % 2))
            R_tmp >>= 1

        R_tmp = R_int

        if i ==0:
            return (1 - c%2)
        else:
            return -c%2

    def LTS(i, x_1, y_1):

        b_1 = (y_1 + r_l) 

        for q in output_queue:
            q.put(b_1)

        b = b_1 + sum(input_queue.get() for _ in range(len(output_queue)))

        a_1 = (r_p_l - x_1)

        for q in output_queue:
            q.put(a_1)

        a = a_1 + sum(input_queue.get() for _ in range(len(output_queue)))

        T = (a + b) 

        barrier.wait()

        w1 = LTBits(i, b, r_l_bits_)
        w2 = LTBits(i, a, r_p_l_bits_)

        w3 = T < b

        added_bit = sum_r_rp_bits_

        w4 = carry
        w5 = LTBits(i, T, added_bit)

        w = w1 ^ w2 ^ w3 ^ w4 ^ w5

        return w

    res = LTS(i, x, y)
    return res

##########################################################################################

def SPDZ_prepare(beta1, beta2, group_sizes, bit_num):
    preparation_data = []

    for group_size in group_sizes:
        # The original shares
        U = random.randint(3, 6)
        V = random.randint(3, 6)
        W = U * V
        U_shares = share(U, group_size)
        V_shares = share(V, group_size)
        W_shares = share(W, group_size)
        x_shares = share(beta1, group_size)
        y_shares = share(beta2, group_size)

        # Additional shares 
        r_l_bits = [0, 0] + [random.randint(0, 1) for _ in range(bit_num - 2)]
        r_l_input = calculate_r_prime(r_l_bits)
        r_l = share(r_l_input, group_size)
        r_l_bits_ = secret_share_binary_list(r_l_bits, group_size)

        r_p_l_bits = [0, 0] + [random.randint(0, 1) for _ in range(bit_num - 2)]
        r_p_l_input = calculate_r_prime(r_p_l_bits)
        r_p_l = share(r_p_l_input, group_size)
        r_p_l_bits_ = secret_share_binary_list(r_p_l_bits, group_size)

        sum_r_rp_bits, carry_ = bitwise_add_and_carry(r_l_bits, r_p_l_bits)
        sum_r_rp_bits_ = secret_share_binary_list(sum_r_rp_bits, group_size)
        carry = share_bit(carry_, group_size)

        # A2B
        rand_bit = random.randint(0, 1)
        rand_bit_p = share(rand_bit, group_size)

        # PremulC
        r_input = [random.randint(1, 5) for _ in range(bit_num)]
        s_input = [random.randint(1, 5) for _ in range(bit_num)]
        r_ = share_beta(r_input, group_size)
        s_ = share_beta(s_input, group_size)

        r_p_mod2_input = [random.randint(0, 1) for _ in range(bit_num)]
        r_p_mod2_share = share_beta(r_p_mod2_input, group_size)

        r_p_mod2_num_input = calculate_r_prime(r_p_mod2_input)
        r_p_mod2_num = share(r_p_mod2_num_input, group_size)
        r_p_mod2_bit_0 = share(r_p_mod2_input[bit_num-1], group_size)

        r_dp_mod2_1_input = [random.randint(0, 1) for _ in range(bit_num)]
        r_dp_mod2_input = calculate_r_prime(r_dp_mod2_1_input)
        r_dp_mod2_1 = share(r_dp_mod2_input, group_size)

        # Append all shares to preparation data
        preparation_data.append(
            (U_shares, V_shares, W_shares, x_shares, y_shares,
             r_l, r_l_bits_, r_p_l, r_p_l_bits_, sum_r_rp_bits_, carry,
             rand_bit_p, r_, s_, r_p_mod2_share, r_p_mod2_num, r_p_mod2_bit_0,
             r_dp_mod2_1))
    
    return preparation_data

def SPDZ_execute(preparation_data, client_num, group_sizes):
    results = []
    all_selected_clients = []
    with concurrent.futures.ThreadPoolExecutor() as executor:
        for group_idx, group_size in enumerate(group_sizes):
            barrier = Barrier(group_size)
            (U_shares, V_shares, W_shares, x_shares, y_shares, 
             r_l, r_l_bits_, r_p_l, r_p_l_bits_, sum_r_rp_bits_, carry, 
             rand_bit_p, r_, s_, r_p_mod2_share, r_p_mod2_num, r_p_mod2_bit_0, 
             r_dp_mod2_1) = preparation_data[group_idx]
            selected_clients = random.sample([i for i in range(client_num) if i not in all_selected_clients], group_size)
            all_selected_clients.extend(selected_clients)
            queues = [queue.Queue() for _ in range(group_size)]
            print(f'Group {group_idx+1}: Threads {selected_clients}')
            futures = [executor.submit(CP, j, x_shares[j], y_shares[j], U_shares[j], 
                                        V_shares[j], W_shares[j], r_l[j], r_l_bits_[j], r_p_l[j], 
                                        r_p_l_bits_[j], sum_r_rp_bits_[j], carry[j], rand_bit_p[j], r_[j], 
                                        s_[j], r_p_mod2_share[j], r_p_mod2_num[j], r_p_mod2_bit_0[j], 
                                        r_dp_mod2_1[j], queues[j], 
                                        [queues[k] for k in range(group_size) if k != j],barrier) 
                       for j in range(group_size)]
            group_results = [f.result() for f in futures]
            results.append(group_results)
            
    for idx, group_result in enumerate(results):
        xor_result = reduce(operator.xor, group_result)
        print(f'Group {idx+1} Results: {group_result}, XOR: {xor_result}')

    output_results = [reduce(operator.xor, group) for group in results]
    print(f'Output Results: {results}, XORs: {output_results}')
    return reduce(operator.xor, output_results)

##############################################################################################

beta1 = 20
beta2 = 10

client_num = 5
group_sizes = generate_groups(client_num, 5, 20, 5)
print("client_num =", client_num)
print("group_sizes =", group_sizes)
print("Total members in groups:", sum(group_sizes))

preparation_data = SPDZ_prepare(beta1, beta2, group_sizes, bit_num)

result = SPDZ_execute(preparation_data, client_num, group_sizes)


client_num = 5
group_sizes = [5]
Total members in groups: 5
Group 1: Threads [0, 1, 3, 4, 2]
Group 1 Results: [0, 0, 0, 1, 1], XOR: 0
Output Results: [[0, 0, 0, 1, 1]], XORs: [0]
